<a href="https://colab.research.google.com/github/na2027/CE888/blob/main/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np


In [10]:
col_names = ['JokeCount'] + [f'Joke{num + 1}' for num in range(100)]
df = pd.read_excel('jester-data-1.xls',header=None, names=col_names)


In [11]:
df.head()

,JokeCount,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,74,-7.82,8.79,-9.66,-8.16,-7.52,-8.50,-9.85,4.17,-8.98,-4.76,-8.50,-6.75,-7.18,8.45,-7.18,-7.52,-7.43,-9.81,-9.85,-9.85,-9.37,1.50,-4.37,-9.81,-8.50,1.12,7.82,2.86,9.13,-7.43,2.14,-4.08,-9.08,7.82,5.05,4.95,-9.17,-8.40,-8.40,...,8.59,3.59,-6.84,-9.03,2.82,-1.36,-9.08,8.30,5.68,-4.81,99.0,99.00,99.0,99.00,99.00,99.00,99.00,-9.42,99.00,99.00,99.00,-7.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,2.82,99.00,99.00,99.00,99.00,99.00,-5.63,99.00,99.00,99.00
1,100,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.0,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
2,49,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.0,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
3,48,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.0,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
4,91,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.0,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60


In [36]:
print(df.loc[11740, 'Joke29'])

-2.04


In [28]:
df.shape

(24983, 100)

In [12]:
df = df.drop(columns=['JokeCount'])


In [13]:
arr = df.values

Labeling 10% of the dataset cells as NaNs (represented as 99 in our case) to create a validation set.


In [16]:
new_arr, validation_indices = replace(arr, 0.1)


In [15]:
def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

3. Infer hidden ratings of the users and hidden characteristics of the jokes by utilizing latent factors modeling.


In [17]:
n_latent_factors = 2

user_ratings = new_arr
# Initialise as random values
latent_user_preferences = np.random.random((user_ratings.shape[0], n_latent_factors))
latent_item_features = np.random.random((user_ratings.shape[1], n_latent_factors))

In [18]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)

In [19]:
def train(user_id, item_id, rating, alpha=0.0001):
    
    #print item_id
    prediction_rating = predict_rating(user_id, item_id)
    err =  prediction_rating - rating
    #print err
    user_pref_values = latent_user_preferences[user_id][:]
    latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
    latent_item_features[item_id] -= alpha * err * user_pref_values
    return err

In [20]:
def sgd(iterations=50):
    """ Iterate over all users and all items and train for 
        a certain number of iterations
    """
    for iteration in range(iterations):
        error = []
        for user_id in range(latent_user_preferences.shape[0]):
            for item_id in range(latent_item_features.shape[0]):
                rating = user_ratings[user_id][item_id]
                if rating != 99:
                    err = train(user_id, item_id, rating)
                    error.append(err)
        mse = (np.array(error) ** 2).mean()   
        if (iteration % 10) == 0:
            print(mse)

In [21]:
sgd()

26.21618978648504
23.29328825672223
21.16196526170869
18.29527925237508
17.336137372805794


4. Comparing predictions (obtained through latent factors modeling) with true values (original dataset).

In [22]:
predictions = latent_user_preferences.dot(latent_item_features.T)
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = [f'Joke{num + 1}' for num in range(100)]

In [29]:
comparison_data

,Joke1,Joke2,Joke3,Joke4,Joke5,Joke6,Joke7,Joke8,Joke9,Joke10,Joke11,Joke12,Joke13,Joke14,Joke15,Joke16,Joke17,Joke18,Joke19,Joke20,Joke21,Joke22,Joke23,Joke24,Joke25,Joke26,Joke27,Joke28,Joke29,Joke30,Joke31,Joke32,Joke33,Joke34,Joke35,Joke36,Joke37,Joke38,Joke39,Joke40,...,Joke61,Joke62,Joke63,Joke64,Joke65,Joke66,Joke67,Joke68,Joke69,Joke70,Joke71,Joke72,Joke73,Joke74,Joke75,Joke76,Joke77,Joke78,Joke79,Joke80,Joke81,Joke82,Joke83,Joke84,Joke85,Joke86,Joke87,Joke88,Joke89,Joke90,Joke91,Joke92,Joke93,Joke94,Joke95,Joke96,Joke97,Joke98,Joke99,Joke100
0,"(-7.82, -3.4798232176801145)","(8.79, -4.074248058429656)","(-9.66, -5.1143358769980685)","(-8.16, -8.599920157837161)","(-7.52, -2.594232926495408)","(99.0, -1.664024635197994)","(-9.85, -4.529195657858374)","(99.0, -4.1161422997869375)","(-8.98, -7.784968304097881)","(99.0, -3.3393871112371762)","(-8.5, -1.5684990127169098)","(99.0, -1.7881299886149726)","(-7.18, -6.343373580260709)","(8.45, -1.774057735620554)","(-7.18, -6.149045189485745)","(99.0, -9.370137240151008)","(-7.43, -4.314908863472704)","(99.0, -4.863296734082605)","(-9.85, -3.4921386407713304)","(-9.85, -5.7799443408133016)","(-9.37, -0.020856205123760914)","(1.5, -4.116363209440647)","(-4.37, -5.647461497702063)","(-9.81, -9.454918128004362)","(-8.5, -4.435533847220891)","(1.12, -1.8814446085030139)","(7.82, 2.9645600075669907)","(2.86, -1.6923571607479293)","(99.0, 2.971667048296004)","(-7.43, -7.064371149126449)","(2.14, 0.7052572416519725)","(-4.08, 2.921342929240597)","(-9.08, -8.467123187856108)","(7.82, -2.8494256140477594)","(5.05, 2.4066851962012934)","(4.95, 3.3147242581222573)","(-9.17, -8.261939237907047)","(99.0, -3.158787228801362)","(-8.4, -2.6139808299843734)","(-8.4, -3.4723025145639794)",...,"(8.59, 0.894946922183561)","(3.59, 1.908758698464276)","(-6.84, -4.8590731085142576)","(-9.03, -7.459192279210506)","(2.82, 0.8552746760830062)","(-1.36, 1.2644003890029416)","(-9.08, -7.947396174149329)","(8.3, 1.5473815890506546)","(5.68, 1.35330414142393)","(-4.81, -4.3714653683528475)","(99.0, -5.374034796033769)","(99.0, 1.3977652833476648)","(99.0, -2.529123329732961)","(99.0, -7.769022871030407)","(99.0, -5.05277133617617)","(99.0, 0.8248065896489885)","(99.0, -3.1304707424396105)","(99.0, -1.3123537796461042)","(99.0, -4.205988933247383)","(99.0, -2.0862882649000305)","(99.0, -0.9022699419355955)","(-7.72, -3.460059053373026)","(99.0, 0.10243151376147651)","(99.0, -3.748002122423333)","(99.0, -3.279118128429272)","(99.0, -5.012648756792839)","(99.0, -0.5970816262050384)","(99.0, -0.5662094463420325)","(99.0, 3.2908461948031857)","(99.0, -4.126259671157238)","(2.82, -0.7530947421219659)","(99.0, -2.4122806217549697)","(99.0, 0.6149907333361924)","(99.0, -3.5152561056277842)","(99.0, -2.923099016486683)","(99.0, -1.9147447030839655)","(-5.63, -1.4122928032947863)","(99.0, -2.0733448504373575)","(99.0, -5.0932541792842905)","(99.0, -2.113673121647738)"
1,"(4.08, 3.0496303165993957)","(-0.29, 3.392155021271348)","(6.36, 3.853754403616923)","(4.37, 5.099351301464294)","(-2.38, 2.416827728954462)","(-9.66, 2.811868275585533)","(-0.73, 2.9977759186461097)","(-5.34, 2.361668622017242)","(99.0, 4.88433724693781)","(9.22, 3.133097321660719)","(6.75, 2.7355996287537483)","(8.64, 2.9426270474637515)","(4.42, 3.2032590376577526)","(7.43, 2.9450644639321695)","(4.56, 2.9151998174944036)","(-0.97, 4.109151264162843)","(4.66, 2.5219003044893333)","(99.0, 2.9227714266598688)","(3.3, 2.818486855561405)","(-1.21, 3.649651232483094)","(0.87, 2.1955008533500906)","(8.64, 3.7571105745086806)","(8.35, 4.150357180915397)","(9.17, 5.242480398952315)","(0.05, 3.7831628256473024)","(7.57, 2.929992725835331)","(4.71, 0.7784171605485105)","(0.87, 2.746452090805026)","(-0.39, 0.6637551761176682)","(6.99, 4.8085882889474645)","(6.5, 1.7792871769726697)","(-0.92, 0.7881862301725282)","(7.14, 5.037856839728481)","(9.03, 3.1879141910018354)","(-1.8, 1.0673893802148056)","(0.73, 0.5264426882352136)","(7.09, 4.684332422039831)","(3.4

In [34]:
print(comparison_data.loc[11740, 'Joke29'])



(-2.04, 0.39847064330927323)


In [23]:
original_data = arr
assert len(validation_indices[0]) == len(validation_indices[1])
validation_size = len(validation_indices[0])
errors = []
for i in range(validation_size):
  user_id = validation_indices[0][i]
  joke_id = validation_indices[1][i]
  prediction_rating = predict_rating(user_id, joke_id)
  real_rating = original_data[user_id][joke_id]
  error = prediction_rating - real_rating
  errors.append(error)
mse = (np.array(errors) ** 2).mean()
print('Validation MSE:', mse)

Validation MSE: 18.11157187076315


Making predictions on the test set.


In [24]:
filled_data = np.zeros(original_data.shape)
rows, columns = original_data.shape
for user_id in range(rows):
  for joke_id in range(columns):
    if original_data[user_id][joke_id] == 99:
      filled_data[user_id][joke_id] = predict_rating(user_id, joke_id)
    else:
      filled_data[user_id][joke_id] = original_data[user_id][joke_id]


In [25]:
filled_data

array([[-7.82      ,  8.79      , -9.66      , ..., -2.07334485,
        -5.09325418, -2.11367312],
       [ 4.08      , -0.29      ,  6.36      , ...,  0.34      ,
        -4.32      ,  1.07      ],
       [ 6.06397319,  6.37726143,  6.3676432 , ...,  6.79120284,
         5.83135753,  7.7173415 ],
       ...,
       [ 0.1429185 ,  0.12903026,  0.07516207, ...,  0.25362237,
         0.04781613,  0.30064629],
       [-1.54027288, -1.83100152, -2.36102104, ..., -0.79624414,
        -2.37078397, -0.78138088],
       [ 2.43      ,  2.67      , -3.98      , ...,  3.52473905,
         2.26003747,  4.06150889]])

Finding the best and worst rated jokes.

In [26]:
means = np.mean(filled_data, axis=0)

In [27]:
max_joke = -9999999999
max_joke_index = -1
min_joke = 99999999999
min_joke_index = -1

for i in range(len(means)):
  if means[i] > max_joke:
    max_joke = means[i]
    max_joke_index = i
  elif means[i] < min_joke:
    min_joke = means[i]
    min_joke_index = i

print(f'The best-rated joke was joke {max_joke_index + 1}, with a mean rating of: {max_joke}')
print(f'The worst-rated joke was joke {min_joke_index + 1}, with a mean rating of: {min_joke}')

The best-rated joke was joke 50, with a mean rating of: 3.665374808969385
The worst-rated joke was joke 58, with a mean rating of: -3.7060720296590746
